In [1]:
from sys import getsizeof
from datetime import datetime
import sys, inspect, dis, asyncio
from functools import wraps
from unittest.mock import MagicMock
import random

jsonify= MagicMock()
hybrid_property= MagicMock()
pytest = MagicMock()
Flask = MagicMock()
cython = MagicMock()
click = MagicMock()
profilehooks = MagicMock()
jit = MagicMock()
numba = MagicMock()

# file touch
with open('hello_world.txt', 'a') as f:
    pass

def null_it(func):
    """Replace with no-op"""
    def null(*args,**kwargs): pass
    return null

prepare = lambda:None
cleanup = lambda:None
expose = lambda x: lambda x: None

# A Taxonomy of Decorators: A-E

Andy Fundinger, Senior Engineer

PyTennessee 2019,
February 10,
Nashville, TN

# Introduction 
* Who I am
* What is Bloomberg?
* What is this talk?
  - We use decorators, but when we go to talk about them, it's hard to get them into categories 
  - With a common terminology, we can discuss more easily
* Who is this talk for?  
  - Intermediate developers who can write decorators, but maybe aren't sure when and why
  - Architects who may need to work across teams to implement and manage their design

## Decorator Syntax and Implementation 

Any decorator can be replaced with code in the decorated functions.  However, decorators allow this code to be reused and factored out of the functions.
- Decorators without the @ sign
- Function Decorators
- Class Decorators
- Decorators with arguments
- Decorators written as classes

Decorators generally insert a section of code of arbitrary complexity in a single line.  Debuggers generally skip over this code--for better or worse.

## Basic Syntax
### Decorators without the @ sign

The oldest decorators in Python are @staticmethod and @classmethod.  Dating back to Python 2.2, we used those like this:

In [2]:
class Util(object):
    def wibble():
        print("Wobble")
    wibble = staticmethod(wibble)

### The @ sign

The @ sign was added in 2.4 as a syntactic sugar for this pattern:

In [3]:
class Util(object):
    @staticmethod
    def wibble():
        print("Wobble")
    @null_it
    def disabled():
        pass

We can write our own decorators as long as we accept a function and  return a replacement function.

In [4]:
def null_it(func):
    """Replace with no-op"""
    def null(*args,**kwargs): pass
    return null

In [5]:
class Printer(object):
    @null_it
    def printOut(self, stuff):
        sys.stdout.write(stuff)
    def print_out(self, stuff):
        print(stuff)
              
Printer().print_out('Nonsense')        

Nonsense


In [6]:
Printer().printOut('something')

### Class Decorators

In Python 2.6, we can decorate classes too:

In [7]:
@null_it
class DeadClass:
    pass

DeadClass() is None

True

### More advanced declarations

#### @wraps - impersonate the orginal function

In [8]:
from functools import wraps

def double_arg1(func):
    #@wraps(func)
    def wrapper(arg_one,*args, **kwargs):
        return func(arg_one*2, *args, **kwargs)
    wrapper.__wrapped__ = func
    wrapper.__module__ = func.__module__
    wrapper.__doc__ = func.__doc__
        
    return wrapper 

@double_arg1
def print_x(x):
    print(x)
    
print_x(1)
print_x('Hello')

2
HelloHello


In [9]:
print_x

<function __main__.double_arg1.<locals>.wrapper>

### Decorators with arguments 

Decorators with arguments aren't actually doing anything all that special.  They simply call the function with the arguments and __that__ function returns the actual decorator.

In [5]:
def mult_arg(mult):
    def deco(func):
        def wrapper(arg_one):
            return func(arg_one * mult)
        return wrapper
    return deco

@mult_arg(3)
def print_x(x):
    print(x)
    
print_x(1)
print_x('Hello ')

3
Hello Hello Hello 


Notice, this is two closures, one for the argument (mult) and one for the function (func).  

### Decorators written as classes

Normally we write decorators that are closures, but there's no particular benefit to this.

In [11]:
import random

class trace_it:
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print(args, kwargs)
        return self.func(*args, **kwargs)
        
@trace_it
def rand(min_val, max_val):
    return random.randint(min_val, max_val)
rand(10,30)

(10, 30) {}


10

## A - Argument Changing Decorators 

* add or remove an argument when the function is called
* change the value or type of an argument at call time 

### Problems 

* calling the apparent signature does not actually work
* calling a function for a test requires injecting data to drive the decorator properly

### Example:  [click.option()](https://click.palletsprojects.com/en/5.x/options/)

In [12]:
@click.command()
@click.option('--n', default=1)
def dots(n):
    print('.' * n)
               
if __name__ == '__main__':
    dots()

### Example implementation -- adding the func name in the call

In [4]:
def fn_with_name(func):
    def wrapper(*args, **kwargs):
        func(func.__name__,*args,**kwargs)
    return wrapper
        
@fn_with_name
def self_aware(name, oth):
    print(f'{name}: {oth}')
    
self_aware('here')
        

self_aware: here


### Return Value Decorators
* run the function "normally"
* alter the output

#### Example: [TurboGears @expose('json')](https://turbogears.readthedocs.io/en/latest/cookbook/jsonp.html#json-and-jsonp-rendering)

In [14]:
@expose('json')
def jp(self, **kwargs):
    return dict(hello='World')

### Example implementation -- wrapping results

In [15]:
def composable_result(func):
    def wrapper(*args, **kwargs):
        return {func.__name__:func(*args,**kwargs)}
    return wrapper
        
@composable_result
def process_1(oth):
    return (oth,)

@composable_result
def process_2():
    return 'Constant Value'
    
results = {}
results.update(process_1('stuff'))
results.update(process_2())
results

{'process_1': ('stuff',), 'process_2': 'Constant Value'}

## B - Binding Decorators

* implement the Descriptor Protocol to change how functions behave
* the standard library includes @staticmethod, @classmethod, and @property

### Problems

* creates an alternative to instance methods and attributes
* new language patterns arguably better fitting other languages 
* time shifts otherwise normal exceptions or introduces new ones

### Example: SQLAlchemy Hybrid Properties

In [16]:
class Interval(object):
    # ...
    
    @hybrid_property
    def radius(self):
        return abs(self.length) / 2

    @radius.expression
    def radius(cls):
        return func.abs(cls.length) / 2

\>>> i1.radius

2

\>>> print Session().query(Interval).filter(Interval.radius > 5)

SELECT interval.id AS interval_id, interval.start AS interval_start,
    interval."end" AS interval_end
FROM interval
WHERE abs(interval."end" - interval.start) / :abs_1 > :param_1

### Example implementation -- instance method

In [17]:
class instance_method:
    def __init__(self, func):
        self.func = func
    def __get__(self, inst, cls):
        if inst is None:
            raise TypeError(f'{self.func.__name__} is only valid on instances.')
        return self.func.__get__(inst, cls)
    
class GoodClass:
    @instance_method
    def simple_method(self):
        print('simple_method')
    def normal_method(self):
        print('normal_method')

In [18]:
GoodClass().normal_method()
GoodClass().simple_method()

normal_method
simple_method


In [19]:
GoodClass.normal_method()

TypeError: normal_method() missing 1 required positional argument: 'self'

In [20]:
GoodClass.simple_method()

TypeError: simple_method is only valid on instances.

In [21]:
GoodClass.normal_method

<function __main__.GoodClass.normal_method>

In [22]:
GoodClass.simple_method

TypeError: simple_method is only valid on instances.

## C - Control Flow Decorators

* change whether a function will be called and how many times

### Problems

* a predictable control flow now has a hidden conditional
* a single invocation might now lead to 0, 1, or many executions of the function

### Example: Retry Decorator

In [38]:
from retrying import retry

@retry
def do_something_unreliable():
    if random.randint(0, 10) > 1:
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"

print(do_something_unreliable())

Awesome sauce!


### Example implementation -- infinite retry

In [25]:
def infinite_retry(func):
    def wrapper(*args, **kwargs):
        while True:
            try:
                return func(*args, **kwargs)
            except RuntimeError as e:
                print(e)
    return wrapper

@infinite_retry
def random_fail(max_value):
    ret = random.randint(-100, max_value)
    if ret<0:
        raise RuntimeError("Invalid negative number {ret}".format(ret=ret))
    return ret

random_fail(10)

Invalid negative number -3
Invalid negative number -13
Invalid negative number -64


5

## D - Descriptive Decorators

* add the decorated object to some sort of collection
* this collection will serve some other purpose such as:
  - documentation
  - dispatching
  - plugins

### Problems 

* it's unclear how dispatching will be done as a result of registration
* similarly it's hard to see where the registration is maintained

### Example pytest.marks

In [26]:
@pytest.mark.webtest
def test_send_http():
    pass # perform some webtest test for your app
def test_something_quick():
    pass
def test_another():
    pass
class TestClass(object):
    def test_method(self):
        pass

[afundinger decorator-taxonomy]$ pytest -m webtest


collected 4 items


test_example.py . 

[100%]

### Example:  flask.app.route

In [27]:
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

### Example implementation -- qa list

In [28]:
to_qa=[]
def qa(func):
    to_qa.append(func.__name__)
    return func

@qa
def new_code(): pass

@qa
def refactored_code(): pass

def well_trusted_code(): pass

In [29]:
to_qa

['new_code', 'refactored_code']

In [30]:
new_code()

## E - Execution Decorators 

* reads the method/class code
* may reinterpret the source code to basically not be python

### Problems
* many
* and more

This truly means that the code you wrote is changed--by the decorator--to some other code that is then executed.  It might:
* be analyzed for dependencies
* have objects in the ast swapped out, injected or removed
* be recompiled with different rules 

### Example: numba

In [2]:
@numba.jit
def f(x, y):
    return x + y

### Example implementation -- code replacer

In [32]:
def replacer(old, new):
    def deco(func):
        source = inspect.getsource(func.__code__)
        lines = source.split('\n')
        new_source = lines[1]+'\n'+('\n'.join(lines[2:]).replace(old, new))
        exec(new_source,globals())
        return globals()[func.__name__]
    return deco

In [33]:
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sample: 10


In [34]:
@replacer('b','b*3')
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sample: 26


In [35]:
@replacer('a','a*3')
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sa*3mple: 14


### Other tools available for execution decorators

* bytecode manipulation
* ast manipulation

## Conclusion

Our Taxonomy
* Argument changing
  - @pytest.mark.parametrize
  - @flask.templated
  - @django.views.decorators.gzip.gzip_page
* Binding
  - @variants.primary
  - @pyramid.decorator.reify
* Control flow
  - @functools.lru_cache
  - @django.views.decorators.http.require_http_methods
  - @twisted.internet.defer.inlineCallbacks
* Descriptive
  - @numpy.testing.decorators.setastest
* Execution
  - @cython.locals
* ?

## W - Wrapping Decorators 

* just runs some code around a function call 
* doesn't affect our function at all
* arguably best replaced with context managers

### Problems

* does so little that it is unlikely to actually remain in this category
* forces code to be designed to have function boundaries at potentially unnatural locations just to allow the decorator to be used

### Example:  [profilehooks.timecall()](https://pypi.org/project/profilehooks/)

In [ ]:
@profilehooks.timecall       
def my_function(args, etc):
    pass

## Feedback Please
* Pacing 
  - Syntax vs Examples
  - Examples themselves
* choice of examples
  - Real world examples
  - Example implementations
  - Execution decorator example

## Bytecode manipulation scratch work 

In [ ]:
sample.__code__.co_code

In [ ]:
dis.dis(sample.__code__.co_code)

In [ ]:
source = inspect.getsource(sample.__code__)
source

In [ ]:
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

In [ ]:
source = inspect.getsource(sample.__code__)
source

In [ ]:
lines = source.split('\n')
new_source = lines[0]+'\n'+('\n'.join(lines[1:]).replace('b','b*3'))

In [ ]:
new_source

In [ ]:
glb = globals()
exec(new_source,glb)

In [ ]:
glb['sample'](1,4), sample(1,4)

In [ ]:
dis.dis(sample)